# Exploration and Skill Acquisition in a Major Online Game

* Tom Stafford; University of Sheffield, t.stafford@shef.ac.uk
* Sam Devlin; Digital Creativity Labs, University of York
* Anders Drachen; Digital Creativity Labs, University of York
* Rafet Sifa; Fraunhofer IAIS, Germany

### Tracking skill acquisition across 21,543 players of the first-person shooter game Destiny shows that variability in some – but not all – dimensions of practice enhances learning

In [1]:
import sys
print (sys.version)

3.5.2 (default, Nov 17 2016, 17:05:23) 
[GCC 5.4.0 20160609]


### Poster 140, Poster Session 1, Monarch Suite, Thursday, July 27, 1:20pm – 2:50pm

Paper: https://mindmodeling.org/cogsci2017/papers/0615/index.html  
Full scripts & sample data: https://osf.io/c59n9/  
Poster: http://tomstafford.staff.shef.ac.uk/docs/CogSci17_TomStafford.jpg  
This notebook: https://github.com/tomstafford/destiny/blob/master/variability.ipynb  

## The Data

In [6]:
#libraries
import os #directory and file functions
import socket #machine id
import pandas as pd #data munging 

import numpy as np #number functions
import pylab as plt #graphing functions

print("working directory = " + os.getcwd())
print("Machine = " + socket.gethostname())

working directory = /home/tom/Dropbox/university/expts/destiny_public/destiny
Machine = tom-xps


In [8]:
#----------------- full data not available (sorry!), but we'll load summary data if we're not running locally

local=False
if not local:
    filename='../../destiny/data/validation_dataset/validation-dataset-rldat.csv' #location of full dataset on Tom's machine

df=pd.read_csv(filename)

In [9]:
df.head()

,destinyMembershipId,date,PvPEventCount,totalDeathDistance,activitiesWon,totalKillDistance,deaths,averageLifespan,objectivesCompleted,averageKillDistance,...,assists,resurrectionsReceived,activitiesEntered,score,averageScorePerLife,maximumWeaponLevel,allParticipantsCount,maximumPowerLevel,highestCharacterLevel,winLossRatio
0,4611686018428705658,2015-12-01T00:00:00Z,1,0,22,8023,216,75.723502,32,17.868597,...,90,16,31,93182,429.410138,0,245,0,40,2.444444
1,4611686018428705658,2015-12-02T00:00:00Z,2,0,0,275,17,39.555556,3,16.176471,...,6,0,1,3695,205.277778,0,16,0,40,0.000000
2,4611686018428705658,2015-12-03T00:00:00Z,3,0,1,254,1,193.500000,0,13.368421,...,1,0,1,2505,1252.500000,0,6,0,40,-1.000000
3,4611686018428705658,2015-12-04T00:00:00Z,4,0,18,5873,106,124.130841,0,30.273196,...,33,42,25,103,0.962617,0,150,0,40,2.571429
4,4611686018428705658,2015-12-08T00:00:00Z,5,0,7,2781,104,62.038095,1,17.490566,...,22,3,10,23625,225.000000,0,85,0,40,2.333333
